##Install necessary libraries

In [1]:
%pip install numpy scipy pandas matplotlib seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch torchvision torchaudio


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install numpy pandas scipy matplotlib scikit-learn 


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torch-scatter torch-sparse torch-geometric


Defaulting to user installation because normal site-packages is not writeable
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
  Using cached aiohttp-3.11.11-cp39-cp39-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached frozenlist-1.5.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached multidict-6.1.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (5.0 kB)
  Using cached propcache-0.2.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached yarl-1.18.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (69 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1

##Load Data

In [31]:
import os
import numpy as np
import pickle

# Corrected path for macOS
dataset_path = "/Users/kostasbekis/Emotion_detection/Deap_dataset/deap/data_preprocessed_python"

# List all .dat files in the directory
all_files = [os.path.join(dataset_path, file) for file in os.listdir(dataset_path) if file.endswith(".dat")]

# Initialize lists for storing data and labels
all_data = []
all_labels = []

# Read data and labels from each .dat file
for file in all_files:
    with open(file, 'rb') as f:
        file_data = pickle.load(f, encoding='latin1')  # Use 'latin1' for compatibility
        all_data.append(file_data['data'])
        all_labels.append(file_data['labels'])

# Convert lists to numpy arrays for easier processing
all_data = np.concatenate(all_data, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

print("Combined data shape:", all_data.shape)
print("Combined labels shape:", all_labels.shape)


Combined data shape: (1280, 40, 8064)
Combined labels shape: (1280, 4)


In [32]:
np.save('/Users/kostasbekis/Emotion_detection/Deap_dataset/deap/data_preprocessed_python', all_data)
np.save('/Users/kostasbekis/Emotion_detection/Deap_dataset/deap/data_preprocessed_python', all_labels)


In [33]:
pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [34]:
from sklearn.preprocessing import StandardScaler
import networkx as nx

# Normalize the EEG data across trials
def normalize_data(data):
    scaler = StandardScaler()
    for i in range(data.shape[0]):
        data[i, :, :] = scaler.fit_transform(data[i, :, :].T).T
    return data

# Normalize EEG data
normalized_data = normalize_data(all_data)
print("Data normalized. Shape:", normalized_data.shape)

# Create adjacency matrix for the GNN
# For simplicity, we'll use a fully connected graph with equal weights
n_channels = normalized_data.shape[1]  # Number of EEG channels (40)
adjacency_matrix = np.ones((n_channels, n_channels))  # Fully connected graph
np.fill_diagonal(adjacency_matrix, 1)  # Diagonal elements are self-connections

print("Adjacency matrix shape:", adjacency_matrix.shape)


Data normalized. Shape: (1280, 40, 8064)
Adjacency matrix shape: (40, 40)


In [35]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Reshape the data for GNN (nodes x features)
# We'll take the average across time steps to reduce dimensionality for simplicity
features = np.mean(normalized_data, axis=2)  # Shape: (1280, 40)
print("Features shape:", features.shape)

# Convert labels to one-hot encoding for multi-class classification
labels_categorical = to_categorical(all_labels > 5)  # Threshold 5 for binary classes
print("Labels shape after one-hot encoding:", labels_categorical.shape)

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(features, labels_categorical, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Train Shape: {X_train.shape}, Validation Shape: {X_val.shape}, Test Shape: {X_test.shape}")
print(f"Train Labels Shape: {y_train.shape}")


Features shape: (1280, 40)
Labels shape after one-hot encoding: (1280, 4, 2)
Train Shape: (896, 40), Validation Shape: (192, 40), Test Shape: (192, 40)
Train Labels Shape: (896, 4, 2)


GNN

In [36]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.18.0


In [37]:
# Ensure adjacency matrix has correct dimensions for batching
batch_size = X_train.shape[0]  # Number of training samples
adjacency_matrix_broadcasted = np.repeat(adjacency_matrix[np.newaxis, :, :], batch_size, axis=0)
print("Correct Broadcasted Adjacency Matrix Shape:", adjacency_matrix_broadcasted.shape)


Correct Broadcasted Adjacency Matrix Shape: (896, 40, 40)


In [38]:
# Add a feature dimension to the node features
X_train = np.expand_dims(X_train, axis=-1)  # New shape: (896, 40, 1)
X_val = np.expand_dims(X_val, axis=-1)      # New shape: (192, 40, 1)
X_test = np.expand_dims(X_test, axis=-1)    # New shape: (192, 40, 1)

print("Updated Inputs shape:", X_train.shape)  # Should now be (896, 40, 1)


Updated Inputs shape: (896, 40, 1)


In [39]:
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_val_tensor = tf.convert_to_tensor(X_val, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)


In [40]:
adjacency_matrix_tensor = tf.convert_to_tensor(adjacency_matrix[:X_train.shape[0]], dtype=tf.float32)  # Training adjacency matrix
adjacency_matrix_val_tensor = tf.convert_to_tensor(adjacency_matrix[:X_val.shape[0]], dtype=tf.float32)  # Validation adjacency matrix
adjacency_matrix_test_tensor = tf.convert_to_tensor(adjacency_matrix[:X_test.shape[0]], dtype=tf.float32)  # Test adjacency matrix


In [41]:
# Define the batch size explicitly
batch_size = X_train.shape[0]  # Number of training samples
n_nodes = features.shape[1]   # Number of nodes (40 in this case)
n_features = 1                # Number of features per node (can adjust if needed)

In [42]:
print(f"Inputs shape: {X_train.shape}, Adjacency matrix shape: {adjacency_matrix_broadcasted.shape}")


Inputs shape: (896, 40, 1), Adjacency matrix shape: (896, 40, 40)


In [43]:
print("X_train_tensor shape:", X_train_tensor.shape)
print("adjacency_matrix_tensor shape:", adjacency_matrix_tensor.shape)
print("y_train shape:", y_train.shape)


X_train_tensor shape: (896, 40, 1)
adjacency_matrix_tensor shape: (40, 40)
y_train shape: (896, 4, 2)


In [44]:
# Expand the adjacency matrix to match the batch size
adjacency_matrix_broadcasted = np.expand_dims(adjacency_matrix, axis=0)  # Shape: (1, 40, 40)
adjacency_matrix_tensor = np.repeat(adjacency_matrix_broadcasted, X_train_tensor.shape[0], axis=0)  # Shape: (896, 40, 40)

# Convert to TensorFlow tensor
adjacency_matrix_tensor = tf.convert_to_tensor(adjacency_matrix_tensor, dtype=tf.float32)

print("Updated adjacency_matrix_tensor shape:", adjacency_matrix_tensor.shape)


Updated adjacency_matrix_tensor shape: (896, 40, 40)


In [45]:
print("Inputs shape:", X_train_tensor.shape)  # Should be (batch_size, 40, 1)
print("Adjacency matrix shape:", adjacency_matrix_tensor.shape)  # Should be (batch_size, 40, 40)
print("Labels shape:", y_train.shape)  # Should be (batch_size, 4, 2)


Inputs shape: (896, 40, 1)
Adjacency matrix shape: (896, 40, 40)
Labels shape: (896, 4, 2)


In [46]:
import numpy as np

print("Node features contain None:", np.any(np.isnan(X_train)))
print("Adjacency matrix contains None:", np.any(np.isnan(adjacency_matrix_tensor)))


Node features contain None: False
Adjacency matrix contains None: False


In [47]:
print("Adjacency matrix is symmetric:", np.allclose(adjacency_matrix, adjacency_matrix.T))
print("Diagonal elements are 1:", np.all(np.diag(adjacency_matrix) == 1))


Adjacency matrix is symmetric: True
Diagonal elements are 1: True


In [48]:
batch_size = X_train.shape[0]
adjacency_matrix_tensor = np.repeat(adjacency_matrix[np.newaxis, :, :], batch_size, axis=0)


In [49]:
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)
print("Test data shape:", X_test.shape)
print("Adjacency matrix shape:", adjacency_matrix_tensor.shape)


Training data shape: (896, 40, 1)
Validation data shape: (192, 40, 1)
Test data shape: (192, 40, 1)
Adjacency matrix shape: (896, 40, 40)


In [50]:
# Example node features
node_features = np.random.rand(896, 40, 1).astype(np.float32)  # Batch of 896 graphs, 40 nodes, 1 feature per node


In [51]:
# Example adjacency matrix
adjacency_matrix = np.ones((896, 40, 40), dtype=np.float32)  # Batch of 896 graphs, fully connected


In [52]:
node_features = np.reshape(node_features, (batch_size, n_nodes, n_features))
adjacency_matrix = np.reshape(adjacency_matrix, (batch_size, n_nodes, n_nodes))


In [53]:
node_features = node_features[:10]
adjacency_matrix = adjacency_matrix[:10]


In [54]:
import os

# Set the environment variable
os.environ["TF_USE_LEGACY_KERAS"] = "1"

# Now import TensorFlow
import tensorflow as tf

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.18.0


In [55]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Legacy Keras enabled:", tf.keras.__version__.startswith("2"))


TensorFlow version: 2.18.0
Legacy Keras enabled: False


In [ ]:
import tensorflow as tf
import tensorflow_gnn as tfgnn
import numpy as np

# Dummy data
batch_size = 8
n_nodes = 40
n_features = 1

# Node features: Shape (batch_size, n_nodes, n_features)
node_features = np.random.rand(batch_size, n_nodes, n_features).astype(np.float32)

# Adjacency matrix: Shape (batch_size, n_nodes, n_nodes)
adjacency_matrix = np.eye(n_nodes, dtype=np.float32)  # Identity matrix for simplicity
adjacency_matrix = np.repeat(adjacency_matrix[np.newaxis, :, :], batch_size, axis=0)

# Build a graph schema
def build_graph_tensor(node_features, adjacency_matrix):
    num_nodes = node_features.shape[1]

    # Flatten batch for TensorFlow GNN
    batch_size = node_features.shape[0]
    node_features_flat = node_features.reshape(-1, n_features)

    # Build adjacency list from adjacency matrix
    source_nodes, target_nodes = np.where(adjacency_matrix[0] > 0)  # Use the first graph as an example
    source_nodes = np.tile(source_nodes, batch_size)  # Repeat for batch
    target_nodes = np.tile(target_nodes, batch_size)

    # Create TensorFlow GNN GraphTensor
    graph = tfgnn.GraphTensor.from_pieces(
        node_sets={
            "nodes": tfgnn.NodeSet.from_fields(
                features={"feature": tf.convert_to_tensor(node_features_flat)},
                sizes=tf.convert_to_tensor([num_nodes] * batch_size)
            )
        },
        edge_sets={
            "edges": tfgnn.EdgeSet.from_fields(
                sizes=tf.convert_to_tensor([len(source_nodes)]),
                adjacency=tfgnn.Adjacency.from_indices(
                    source=("nodes", tf.convert_to_tensor(source_nodes, dtype=tf.int32)),
                    target=("nodes", tf.convert_to_tensor(target_nodes, dtype=tf.int32))
                )
            )
        }
    )
    return graph

# Build graph tensor
graph_tensor = build_graph_tensor(node_features, adjacency_matrix)

# Define the GNN model using TensorFlow GNN
class GCNModel(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.gnn_layer = tfgnn.keras.layers.GraphUpdate(
            node_sets={"nodes": tfgnn.keras.layers.SimpleConv(units, activation="relu")}
        )
        self.dense = tf.keras.layers.Dense(units, activation="relu")

    def call(self, graph_tensor):
        graph_tensor = self.gnn_layer(graph_tensor)
        node_features = graph_tensor.node_sets["nodes"]["feature"]
        return self.dense(node_features)

# Instantiate and compile the model
units = 32
model = GCNModel(units)
model.compile(optimizer="adam", loss="mse")  # Adjust loss as needed

# Forward pass
output = model(graph_tensor)
print("Output shape:", output.shape)


ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
history = model.fit(
    [X_train, adjacency_matrix_broadcasted[:X_train.shape[0]]],  # Features and adjacency matrix for training
    [y_train[:, 0, :], y_train[:, 1, :], y_train[:, 2, :], y_train[:, 3, :]],  # Separate outputs
    validation_data=(
        [X_val, adjacency_matrix_broadcasted[:X_val.shape[0]]],  # Validation data
        [y_val[:, 0, :], y_val[:, 1, :], y_val[:, 2, :], y_val[:, 3, :]]  # Validation labels
    ),
    epochs=20,
    batch_size=32
)


In [ ]:
# Evaluate the model
loss, valence_acc, arousal_acc, dominance_acc, liking_acc = model.evaluate(
    [X_test, adjacency_matrix_broadcasted[:X_test.shape[0]]],
    [y_test[:, 0, :], y_test[:, 1, :], y_test[:, 2, :], y_test[:, 3, :]]
)

print(f"Valence Accuracy: {valence_acc:.2f}")
print(f"Arousal Accuracy: {arousal_acc:.2f}")
print(f"Dominance Accuracy: {dominance_acc:.2f}")
print(f"Liking Accuracy: {liking_acc:.2f}")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predictions
y_pred = model.predict([X_test, adjacency_matrix_broadcasted[:X_test.shape[0]]])

# Process predictions and ground truth
y_pred_classes = [np.argmax(output, axis=-1) for output in y_pred]
y_test_classes = [np.argmax(output, axis=-1) for output in [y_test[:, 0, :], y_test[:, 1, :], y_test[:, 2, :], y_test[:, 3, :]]]

# Confusion matrix for each output
for i, label_name in enumerate(["Valence", "Arousal", "Dominance", "Liking"]):
    cm = confusion_matrix(y_test_classes[i], y_pred_classes[i])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix: {label_name}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

    print(f"Classification Report for {label_name}:")
    print(classification_report(y_test_classes[i], y_pred_classes[i]))


In [ ]:
import tensorflow as tf
import spektral

print("TensorFlow version:", tf.__version__)
print("Spektral version:", spektral.__version__)


ModuleNotFoundError: No module named 'tensorflow'